**bnt-biftool**

This notebook defines some functions that enhance BNT (Bayes  Net Toolbox, by Kevin Murphy et al).
They allow BNT to read and write bif and dot files. These functions only work
for bnets whose nodes all have discrete PDs. No need to worry about topological ordering 
of nodes. This notebook takes care of it.

This notebook is intended for use inside another notebook running on a Python
kernel with octavemagic loaded. It also assumes that (1) the QFog modules
BayesNet and its dependencies have been imported. (2) the BNT files mk_bnet.m and
tabular_CPD.m and dependencies are in the path variable of Octave. 

This notebook has been used and tested within the notebook ```bnt-read-write-a-bif.ipynb```.

This notebook depends on more than just QFog's BifTool. It actually creates an interim QFog BayesNet to take advantage of the topological sort of QFog.


# Table of Contents
 <p><div class="lev1 toc-item"><a href="#BNT-reads" data-toc-modified-id="BNT-reads-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>BNT reads</a></div><div class="lev1 toc-item"><a href="#BNT-writes" data-toc-modified-id="BNT-writes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>BNT writes</a></div>

# BNT reads

In [1]:
def bnt_push_bnet(bn_fog, do_pots, verbose=False):
    """

    Parameters
    ----------
    bn_fog : BayesNet
    do_pots : bool
        whether to deal with pots or not (True for bif files and False for dot files)
    verbose : bool

    Returns
    -------
    None

    """
    if verbose:
        print("*****************inside push_bnet*****************")
    global adj_mat, nd_sizes, nd_names, num_nds
    global nd_num, nd_name, flat_arr

    num_nds = bn_fog.num_nodes
    adj_mat = np.zeros((num_nds, num_nds), dtype=int)
    # print("------99999999999")
    # print(adj_mat)
    nd_sizes = [0]*num_nds
    nd_names = ['a']*num_nds
    for nd in bn_fog.nodes:
        pa_indices = [pa.topo_index for pa in nd.parents]
        adj_mat[pa_indices, nd.topo_index] = 1
        nd_sizes[nd.topo_index] = nd.size
        nd_names[nd.topo_index] = nd.name

    # print(adj_mat)
    %octave_push adj_mat nd_sizes nd_names num_nds
    %octave bnet = mk_bnet(adj_mat, nd_sizes, 'names', nd_names, 'discrete', 1:num_nds);

    if do_pots:
        for nd in bn_fog.nodes:
            nd_num = nd.topo_index + 1
            nd_name = nd.name
            pot = nd.potential
            topo_indices = [x.topo_index for x in pot.ord_nodes]
            #calculate permutation required to put topo_indices in increasing order
            perm =  [i[0] for i in sorted(enumerate(topo_indices), key=lambda x:x[1])]
            flat_arr = np.transpose(pot.pot_arr, perm).flatten(order='F')

            %octave_push nd_num flat_arr
            %octave bnet.CPD{1, nd_num} = tabular_CPD(bnet, nd_num, 'CPT', flat_arr);
            if verbose:
                print("------------pot for " + nd.name + "------------")
                print('topo_indices=', topo_indices)
                print('perm=', perm)
                print(pot)
                print('flattened=', flat_arr)
                %octave bnet.CPD{1, nd_num}


def bnt_read_bif(in_path, verbose=False):
    bn_fog = BayesNet.read_bif(in_path, False)
    bnt_push_bnet(bn_fog, True, verbose=verbose)
    return bn_fog.get_vtx_to_state_names()


def bnt_read_dot(in_path, verbose=False):
    bn_fog = BayesNet.read_dot(in_path)
    bnt_push_bnet(bn_fog, False, verbose=verbose)


# BNT writes

In [2]:
def bnt_pull_bnet(do_pots, vtx_to_states=None, verbose=False):
    """
    This function expects on entry for a well-formed BNT bnet = mk_bnet() to be in the 
    octave memory. We don't pass it in as a parameter because it is an octave object
    instead of a python one.

    Parameters
    ----------
    do_pots : bool
        whether to deal with pots or not (True for bif files and False for dot files)
    vtx_to_states : dict[str, list[str]]
        A dictionary mapping each node name to a list of the names of its states. 
        BNT doesn't store state names. You can pass them in with this variable
        so that they are used in writing the bif file. If vtx_to_states=None, 
        node states will be named state0, state1, state2, ...
    verbose : bool

    Returns
    -------
    BayesNet

    """
    global kk
    if verbose:
        print("*****************inside pull_bnet*****************")
    # %octave disp(bnet)
    adj_mat = %octave bnet.dag;
    adj_mat = adj_mat.astype(int)
    # print(adj_mat)
    
    num_nds = adj_mat.shape[0]

    nd_sizes = %octave bnet.node_sizes;
    nd_sizes = list(nd_sizes[0].astype(int))
    # print(nd_sizes)

    nd_names = %octave struct(bnet.names);
    nd_names = list(nd_names['keys'][0])
    # print(nd_names)

    node_list = []
    for k in range(num_nds):
        nd = BayesNode(k, nd_names[k])
        node_list.append(nd)
    nd_to_parents = {}
    for k2, nd2 in enumerate(node_list):
        nd_to_parents[nd2] = []
        # From tests, BNT does not always return topologically ordered adj_mat.
        # If it did, adj_mat would be lower triangular and could use 
        # for k1 in range(k2)
        for k1 in range(num_nds):
            nd1 = node_list[k1]
            if adj_mat[k1, k2] == 1:
                nd2.add_parent(nd1)
                # parents in nd_to_parents[nd2] 
                # must be in increasing k1 order
                nd_to_parents[nd2].append(nd1)
    bn_fog = BayesNet(set(node_list))
    
    if vtx_to_states:
        bn_fog.import_nd_state_names(vtx_to_states)

    if do_pots:
        for k, nd in enumerate(node_list):
            kk = k+1
            parents = nd_to_parents[nd]
            if not parents:
                nd.potential = DiscreteUniPot(False, nd)
            else:
                nd.potential = DiscreteCondPot(False, parents + [nd])
                
            %octave_push kk                
            # nd.potential.pot_arr = %octave get_field(bnet.CPD{kk}, 'cpt') 
            
            # this is necessary because current oct2py has bug in parsing cell assignments
            %octave x = bnet.CPD{1, kk};
            %octave pos = 1;
            %octave y = x{pos};
            %octave st = 'cpt';                     
            %octave z = get_field(y, st);
            %octave_pull z
             
            # change column vectors to row vectors
            sh = z.shape
            arr = z
            if len(sh) == 2 and sh[1]==1:
                #print(arr)
                arr = z.T[0]       
                #print(arr)           
            nd.potential.pot_arr = arr
            
            if verbose:
                print("------------pot for " + nd.name + "------------")
                print(nd.potential)
                %octave z

    return bn_fog


def bnt_write_bif(out_path, vtx_to_states=None, verbose=False):
    bn_fog = bnt_pull_bnet(True, vtx_to_states=vtx_to_states, verbose=verbose)
    bn_fog.write_bif(out_path, False)


def bnt_write_dot(out_path, verbose=False):
    bn_fog = bnt_pull_bnet(False, verbose=verbose)
    bn_fog.write_dot(out_path)